In [2]:
%pip -q install anthropic

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from dotenv import load_dotenv
from pygments import highlight
from pygments.lexers import JavaLexer
from pygments.formatters import TerminalFormatter
import sqlite3
import pandas as pd
import random
import numpy as np

In [4]:
# Path to your SQLAR database
db_path = 'gcj-archive-2018/2018r3/solutions.sqlar'
# Connect to the SQLAR file
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

In [5]:
df = pd.read_sql("SELECT data , name  FROM sqlar WHERE name LIKE '%.java'", conn)
# look into evaluating code security/ bugs / quality measurements
df['user_id'] = df['name'].str.extract(r'/([A-Za-z0-9]+)\.')
df['data'] = df['data'].str.decode('utf-8')
print("# of authors: " , len(df.groupby('user_id')))
df

# of authors:  33


,data,name,user_id
0,import java.io.*;\r\nimport java.math.*;\r\nim...,solutions/2rf.0.java,2rf
1,import java.io.*;\r\nimport java.math.*;\r\nim...,solutions/2rf.1.java,2rf
2,import java.io.*;\r\nimport java.util.*;\r\n\r...,solutions/2rf.2.java,2rf
3,import java.io.*;\r\nimport java.math.*;\r\nim...,solutions/2rf.3.java,2rf
4,import java.io.*;\r\nimport java.math.*;\r\nim...,solutions/2rf.4.java,2rf
...,...,...,...
134,import java.awt.*;\r\nimport java.awt.geom.*;\...,solutions/xiaowuc1.1.java,xiaowuc1
135,import java.awt.*;\r\nimport java.awt.geom.*;\...,solutions/xiaowuc1.2.java,xiaowuc1
136,import java.awt.*;\r\nimport java.awt.geom.*;\...,solutions/xiaowuc1.3.java,xiaowuc1
137,import java.awt.*;\r\nimport java.awt.geom.*;\...,solutions/xiaowuc1.4.java,xiaowuc1


In [6]:
filter = df.groupby(by='user_id').count() > 2
valid_users = filter[filter['data']].index
df_filtered = df[df['user_id'].isin(valid_users)]
df_filtered

,data,name,user_id
0,import java.io.*;\r\nimport java.math.*;\r\nim...,solutions/2rf.0.java,2rf
1,import java.io.*;\r\nimport java.math.*;\r\nim...,solutions/2rf.1.java,2rf
2,import java.io.*;\r\nimport java.util.*;\r\n\r...,solutions/2rf.2.java,2rf
3,import java.io.*;\r\nimport java.math.*;\r\nim...,solutions/2rf.3.java,2rf
4,import java.io.*;\r\nimport java.math.*;\r\nim...,solutions/2rf.4.java,2rf
...,...,...,...
134,import java.awt.*;\r\nimport java.awt.geom.*;\...,solutions/xiaowuc1.1.java,xiaowuc1
135,import java.awt.*;\r\nimport java.awt.geom.*;\...,solutions/xiaowuc1.2.java,xiaowuc1
136,import java.awt.*;\r\nimport java.awt.geom.*;\...,solutions/xiaowuc1.3.java,xiaowuc1
137,import java.awt.*;\r\nimport java.awt.geom.*;\...,solutions/xiaowuc1.4.java,xiaowuc1


In [7]:
# load api key
load_dotenv()

True

In [8]:
import pandas as pd
import numpy as np
from langchain_community.callbacks import get_openai_callback

import contextlib
import io
from openai import OpenAI


In [9]:
code_snippet_1 = df_filtered.iloc[0, 0]
code_snippet_2 = df_filtered.iloc[3,0]

# print(highlight(code_snippet_1, JavaLexer(), TerminalFormatter())),
# print(highlight(code_snippet_2, JavaLexer(), TerminalFormatter()))

code_snippet_2

'import java.io.*;\r\nimport java.math.*;\r\nimport java.util.*;\r\nimport java.util.stream.*;\r\n\r\npublic class Solution {\r\n\r\n\tvoid add(List<Integer>[] g, int v, int u) {\r\n\t\tg[v].add(u);\r\n\t\tg[u].add(v);\r\n\t}\r\n\r\n\tlong P = BigInteger.probablePrime(45, rng).longValueExact();\r\n\tlong Q = BigInteger.probablePrime(10, rng).longValueExact();\r\n\r\n\tlong[] iterate(long[] hash, List<Integer>[] g) {\r\n\t\tint n = hash.length;\r\n\t\tlong[] next = new long[n];\r\n\t\tfor (int i = 0; i < n; i++) {\r\n\t\t\t\r\n\t\t\tlong[] arr = new long[5];\r\n\t\t\tarr[0] = hash[i];\r\n\t\t\tint ptr = 1;\r\n\t\t\tfor (int u : g[i]) {\r\n\t\t\t\tarr[ptr++] = hash[u];\r\n\t\t\t}\r\n\t\t\tArrays.sort(arr, 1, 5);\r\n\t\t\t\r\n//\t\t\tlong[] arr = new long[4];\r\n//\t\t\tint ptr = 0;\r\n//\t\t\tfor (int u : g[i]) {\r\n//\t\t\t\tarr[ptr++] = hash[u];\r\n//\t\t\t}\r\n//\t\t\tArrays.sort(arr);\r\n\t\t\t\r\n\t\t\tlong val = 0;\r\n\t\t\tfor (long x : arr) {\r\n\t\t\t\tval = (val * Q + x);\r\n\t

In [10]:
prompt = '''
You are a helpful assistant. Your task is to verify if the following two code snippets were written by the same author. Please provide the liklihood (1 indicating likly and 2 indicating unlikely) that code snippet 1 and code snippet 2 were written by the same author, disregarding the functionality difference in code functionality. Here are some relevant variables to this problem.
1. AST Node Bigrams: Count adjacent syntax nodes, capturing structure like ‘if’ followed by a ‘block’.  
2. AST Node Types: Measure frequency of specific syntax elements; e.g., counting loops, conditionals, function declarations.  
3. AST Node Types (TFIDF): Weight rare syntax elements; e.g., unusual recursion node counts boost identification.  
4. Average AST Depth: Quantifies overall code nesting; e.g., deeply nested loops significantly increase average depth.  
5. Keywords Frequency: Counts language keywords usage; e.g., detecting repeated conditional or loop keywords across code.  
6. AST Leaf Code Unigrams: Count tokens from leaf nodes; e.g., identifiers or operators appear repeatedly.  
7. AST Leaf Code Unigrams (TFIDF): Weigh unique leaf tokens; e.g., rarely used identifiers increase importance.  
8. Average Leaf Token Depth: Captures token depth; e.g., deeply nested identifiers suggest complex code structure.  
9. Lexical Word Unigrams: Count tokens from source; e.g., common operators and identifiers indicate stylistic choices.  
10. Whitespace Ratio: Measures proportion of whitespace to characters; e.g., code with spacing versus compact formatting.
'''

prompt_2 = f'''
code snippet 1:
{code_snippet_1}

code snippet 2:
{code_snippet_2}
'''


In [11]:
client = OpenAI()
response = client.chat.completions.create(
    model="o3-mini",
    messages=[
        {"role": "developer", "content": prompt},
        {"role": "user", "content": prompt_2},
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "code_authorship_verification",
            "schema": {
                "type": "object",
                "properties": {
                    "classification": {
                        "description": "Whether or not the 2 code snippets provided are written by the same author",
                        "type": "boolean"
                    },
                    "additionalProperties": False
                }
            }
        }
    }
)

In [12]:
response.choices[0].message.content

'{"classification": true}'

In [19]:
import json

def evaluate_pair(code1, user1, code2, user2, system_prompt=None, input_prompt=None):
    """Evaluate a pair of code snippets and return if prediction was correct"""
    prompt = system_prompt or '''
    You are a helpful assistant. Your task is to verify if the following two code snippets were written by the same author. Please provide the liklihood (1 indicating likly and 2 indicating unlikely) that code snippet 1 and code snippet 2 were written by the same author, disregarding the functionality difference in code functionality. Here are some relevant variables to this problem.
    1. AST Node Bigrams: Count adjacent syntax nodes, capturing structure like ‘if’ followed by a ‘block’.  
    2. AST Node Types: Measure frequency of specific syntax elements; e.g., counting loops, conditionals, function declarations.  
    3. AST Node Types (TFIDF): Weight rare syntax elements; e.g., unusual recursion node counts boost identification.  
    4. Average AST Depth: Quantifies overall code nesting; e.g., deeply nested loops significantly increase average depth.  
    5. Keywords Frequency: Counts language keywords usage; e.g., detecting repeated conditional or loop keywords across code.  
    6. AST Leaf Code Unigrams: Count tokens from leaf nodes; e.g., identifiers or operators appear repeatedly.  
    7. AST Leaf Code Unigrams (TFIDF): Weigh unique leaf tokens; e.g., rarely used identifiers increase importance.  
    8. Average Leaf Token Depth: Captures token depth; e.g., deeply nested identifiers suggest complex code structure.  
    9. Lexical Word Unigrams: Count tokens from source; e.g., common operators and identifiers indicate stylistic choices.  
    10. Whitespace Ratio: Measures proportion of whitespace to characters; e.g., code with spacing versus compact formatting.
    '''

    prompt_2 = f'''
    code snippet 1:
    {code1}

    code snippet 2:
    {code2}
    '''

    client = OpenAI()

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "developer", "content": prompt},
            {"role": "user", "content": prompt_2},
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "code_authorship_verification",
                "schema": {
                    "type": "object",
                    "properties": {
                        "classification": {
                            "description": "Whether or not the 2 code snippets provided are written by the same author",
                            "type": "boolean"
                        },
                        "additionalProperties": False
                    }
                }
            }
        }
    )
    
    print(response.model)
    prediction = json.loads(response.choices[0].message.content)['classification']

    print("[PREDICTION] " , prediction)
    same_author = user1 == user2
    print("[SAME AUTHOR]", same_author)
    return prediction == same_author

In [20]:
# Add these imports at the top
import itertools
import random

def run_experiment(df, eval_fn, num_pairs=200):
    """Run experiment on balanced random pairs of code snippets"""
    
    # Reset DataFrame index to ensure continuous integers
    df = df.reset_index(drop=True)
    
    # Get all unique authors
    authors = df['user_id'].unique()
    
    # Create same-author pairs
    same_author_pairs = []
    for author in authors:
        # Get all submissions from this author
        author_submissions = df[df['user_id'] == author].index.tolist()
        if len(author_submissions) >= 2:
            # Get all possible pairs of submissions from this author
            author_pairs = list(itertools.combinations(author_submissions, 2))
            same_author_pairs.extend(author_pairs)
    
    # Create different-author pairs
    different_author_pairs = []
    for author1, author2 in itertools.combinations(authors, 2):
        # Get submissions from both authors
        author1_submissions = df[df['user_id'] == author1].index.tolist()
        author2_submissions = df[df['user_id'] == author2].index.tolist()
        # Create pairs between their submissions
        pairs = list(itertools.product(author1_submissions, author2_submissions))
        different_author_pairs.extend(pairs)
    
    # Ensure we have enough pairs of each type
    pairs_per_type = min(num_pairs // 2, len(same_author_pairs), len(different_author_pairs))
    if pairs_per_type == 0:
        raise ValueError("Not enough pairs available for evaluation")
    
    # Randomly sample equal numbers of each type
    selected_same_pairs = random.sample(same_author_pairs, pairs_per_type)
    selected_diff_pairs = random.sample(different_author_pairs, pairs_per_type)
    
    # Combine and shuffle
    all_pairs = selected_same_pairs + selected_diff_pairs
    random.shuffle(all_pairs)
    
    correct = 0
    total = len(all_pairs)
    
    print(f"Total pairs to evaluate: {total} ({pairs_per_type} same-author, {pairs_per_type} different-author)")
    
    for i, (idx1, idx2) in enumerate(all_pairs):
        try:
            code1 = df.iloc[idx1]['data']
            code2 = df.iloc[idx2]['data']
            user1 = df.iloc[idx1]['user_id']
            user2 = df.iloc[idx2]['user_id']
            
            print(f"\nEvaluating pair {i+1}/{total}")
            print(f"Authors: {user1}, {user2}")
            
            is_correct = eval_fn(code1, user1, code2, user2)
            correct += is_correct
            print(f"Current accuracy: {correct/(i+1):.2%}")
        except Exception as e:
            print(f"Error processing pair {i+1}: {e}")
            continue
    
    final_accuracy = correct/total if total > 0 else 0
    print(f"\nFinal Accuracy: {final_accuracy:.2%}")
    return final_accuracy

In [21]:

# Run the experiment
accuracy = run_experiment(df_filtered, eval_fn=evaluate_pair)  # Adjust num_pairs as needed

Total pairs to evaluate: 200 (100 same-author, 100 different-author)

Evaluating pair 1/200
Authors: stolis, tmwilliamlin168
gpt-4o-mini-2024-07-18
[PREDICTION]  2
[SAME AUTHOR] False
Current accuracy: 0.00%

Evaluating pair 2/200
Authors: tmwilliamlin168, xiaowuc1
gpt-4o-mini-2024-07-18
[PREDICTION]  2
[SAME AUTHOR] False
Current accuracy: 0.00%

Evaluating pair 3/200
Authors: Yarin, Yarin
gpt-4o-mini-2024-07-18
[PREDICTION]  True
[SAME AUTHOR] True
Current accuracy: 33.33%

Evaluating pair 4/200
Authors: kongbomb, peter707
gpt-4o-mini-2024-07-18
[PREDICTION]  2
[SAME AUTHOR] False
Current accuracy: 25.00%

Evaluating pair 5/200
Authors: andrewzta, bovinophile
gpt-4o-mini-2024-07-18
[PREDICTION]  2
[SAME AUTHOR] False
Current accuracy: 20.00%

Evaluating pair 6/200
Authors: Yarin, Yarin
gpt-4o-mini-2024-07-18
[PREDICTION]  True
[SAME AUTHOR] True
Current accuracy: 33.33%

Evaluating pair 7/200
Authors: peter707, peter707
gpt-4o-mini-2024-07-18
[PREDICTION]  True
[SAME AUTHOR] True
Cur

In [31]:
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.environ['ANTHROPIC_API_KEY'],
)


In [32]:
import os
from dotenv import load_dotenv
from pygments import highlight
from pygments.lexers import JavaLexer
from pygments.formatters import TerminalFormatter

from typing import Optional
from pydantic import BaseModel, Field

from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate

import numpy as np
from langchain_community.callbacks import get_openai_callback
from langchain_core.output_parsers import StrOutputParser

import contextlib

In [33]:
llm = ChatAnthropic(
        model="claude-3-5-sonnet-20241022",
        temperature=0,
        max_tokens_to_sample=4000
    )

# Define the Pydantic model for structured output
class VulnerabilityInjection(BaseModel):
   """
   You are a helpful assistant. Your task is to verify if the following two code snippets were written by the same author. Please provide the liklihood (1 indicating likly and 2 indicating unlikely) that code snippet 1 and code snippet 2 were written by the same author, disregarding the functionality difference in code functionality. Here are some relevant variables to this problem.
    1. AST Node Bigrams: Count adjacent syntax nodes, capturing structure like ‘if’ followed by a ‘block’.  
    2. AST Node Types: Measure frequency of specific syntax elements; e.g., counting loops, conditionals, function declarations.  
    3. AST Node Types (TFIDF): Weight rare syntax elements; e.g., unusual recursion node counts boost identification.  
    4. Average AST Depth: Quantifies overall code nesting; e.g., deeply nested loops significantly increase average depth.  
    5. Keywords Frequency: Counts language keywords usage; e.g., detecting repeated conditional or loop keywords across code.  
    6. AST Leaf Code Unigrams: Count tokens from leaf nodes; e.g., identifiers or operators appear repeatedly.  
    7. AST Leaf Code Unigrams (TFIDF): Weigh unique leaf tokens; e.g., rarely used identifiers increase importance.  
    8. Average Leaf Token Depth: Captures token depth; e.g., deeply nested identifiers suggest complex code structure.  
    9. Lexical Word Unigrams: Count tokens from source; e.g., common operators and identifiers indicate stylistic choices.  
    10. Whitespace Ratio: Measures proportion of whitespace to characters; e.g., code with spacing versus compact formatting.
   """

   classification: bool = Field(description="Whether or not the 2 code snippets provided are written by the same author")

In [34]:
# Define the Pydantic model for structured output
system = """
You are a helpful assistant. Your task is to verify if the following two code snippets were written by the same author. Please provide the liklihood (1 indicating likly and 2 indicating unlikely) that code snippet 1 and code snippet 2 were written by the same author, disregarding the functionality difference in code functionality. Here are some relevant variables to this problem.
    1. AST Node Bigrams: Count adjacent syntax nodes, capturing structure like ‘if’ followed by a ‘block’.  
    2. AST Node Types: Measure frequency of specific syntax elements; e.g., counting loops, conditionals, function declarations.  
    3. AST Node Types (TFIDF): Weight rare syntax elements; e.g., unusual recursion node counts boost identification.  
    4. Average AST Depth: Quantifies overall code nesting; e.g., deeply nested loops significantly increase average depth.  
    5. Keywords Frequency: Counts language keywords usage; e.g., detecting repeated conditional or loop keywords across code.  
    6. AST Leaf Code Unigrams: Count tokens from leaf nodes; e.g., identifiers or operators appear repeatedly.  
    7. AST Leaf Code Unigrams (TFIDF): Weigh unique leaf tokens; e.g., rarely used identifiers increase importance.  
    8. Average Leaf Token Depth: Captures token depth; e.g., deeply nested identifiers suggest complex code structure.  
    9. Lexical Word Unigrams: Count tokens from source; e.g., common operators and identifiers indicate stylistic choices.  
    10. Whitespace Ratio: Measures proportion of whitespace to characters; e.g., code with spacing versus compact formatting.
"""

In [35]:
structured_llm = llm.with_structured_output(
        VulnerabilityInjection)
# Create the prompt
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{input}")])
output_parser = StrOutputParser()
chain = prompt | llm | output_parser

In [36]:
# Invoke the LLM with the original snippet
import json
with get_openai_callback() as cb:
    result = structured_llm.invoke(prompt_2)
    print(result.classification)
    # result_dict = json.loads(result, strict=False)
    # classification = VulnerabilityInjection(**result_dict)
    # print(classification)
# print(f"Total Tokens: {cb.total_tokens}")
print(f"Total Tokens: {cb.total_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")

True
Total Tokens: 5604
Total Cost (USD): $0.0


In [37]:
def eval_pair_anthropic(code1, user1, code2, user2):
    llm = ChatAnthropic(
        model="claude-3-5-sonnet-20241022",
        temperature=0,
        max_tokens_to_sample=4000
    )

    system = """
    You are a helpful assistant. Your task is to verify if the following two code snippets were written by the same author. Please provide the liklihood (1 indicating likly and 2 indicating unlikely) that code snippet 1 and code snippet 2 were written by the same author, disregarding the functionality difference in code functionality. Here are some relevant variables to this problem.
        1. AST Node Bigrams: Count adjacent syntax nodes, capturing structure like ‘if’ followed by a ‘block’.  
        2. AST Node Types: Measure frequency of specific syntax elements; e.g., counting loops, conditionals, function declarations.  
        3. AST Node Types (TFIDF): Weight rare syntax elements; e.g., unusual recursion node counts boost identification.  
        4. Average AST Depth: Quantifies overall code nesting; e.g., deeply nested loops significantly increase average depth.  
        5. Keywords Frequency: Counts language keywords usage; e.g., detecting repeated conditional or loop keywords across code.  
        6. AST Leaf Code Unigrams: Count tokens from leaf nodes; e.g., identifiers or operators appear repeatedly.  
        7. AST Leaf Code Unigrams (TFIDF): Weigh unique leaf tokens; e.g., rarely used identifiers increase importance.  
        8. Average Leaf Token Depth: Captures token depth; e.g., deeply nested identifiers suggest complex code structure.  
        9. Lexical Word Unigrams: Count tokens from source; e.g., common operators and identifiers indicate stylistic choices.  
        10. Whitespace Ratio: Measures proportion of whitespace to characters; e.g., code with spacing versus compact formatting.
    """

    prompt = '''
        You are a helpful assistant. Your task is to verify if the following two code snippets were written by the same author. Please provide the liklihood (1 indicating likly and 2 indicating unlikely) that code snippet 1 and code snippet 2 were written by the same author, disregarding the functionality difference in code functionality. Here are some relevant variables to this problem.
        1. AST Node Bigrams: Count adjacent syntax nodes, capturing structure like ‘if’ followed by a ‘block’.  
        2. AST Node Types: Measure frequency of specific syntax elements; e.g., counting loops, conditionals, function declarations.  
        3. AST Node Types (TFIDF): Weight rare syntax elements; e.g., unusual recursion node counts boost identification.  
        4. Average AST Depth: Quantifies overall code nesting; e.g., deeply nested loops significantly increase average depth.  
        5. Keywords Frequency: Counts language keywords usage; e.g., detecting repeated conditional or loop keywords across code.  
        6. AST Leaf Code Unigrams: Count tokens from leaf nodes; e.g., identifiers or operators appear repeatedly.  
        7. AST Leaf Code Unigrams (TFIDF): Weigh unique leaf tokens; e.g., rarely used identifiers increase importance.  
        8. Average Leaf Token Depth: Captures token depth; e.g., deeply nested identifiers suggest complex code structure.  
        9. Lexical Word Unigrams: Count tokens from source; e.g., common operators and identifiers indicate stylistic choices.  
        10. Whitespace Ratio: Measures proportion of whitespace to characters; e.g., code with spacing versus compact formatting.
        '''

    prompt_2 = f'''
    code snippet 1:
    {code1}

    code snippet 2:
    {code2}
    '''

    structured_llm = llm.with_structured_output(VulnerabilityInjection)
    # Create the prompt
    prompt = ChatPromptTemplate.from_messages(
        [("system", system), ("human", "{input}")])
    output_parser = StrOutputParser()
    chain = prompt | llm | output_parser
    result = structured_llm.invoke(prompt_2)


    prediction = result.classification

    print("[PREDICTION] " , prediction)
    same_author = user1 == user2
    print("[SAME AUTHOR]", same_author)
    return prediction == same_author

In [38]:
# Run the experiment
accuracy = run_experiment(df_filtered, eval_fn=eval_pair_anthropic)  # Adjust num_pairs as needed
accuracy

Total pairs to evaluate: 6 (3 same-author, 3 different-author)

Evaluating pair 1/6
Authors: qwerty787788, uwi
[PREDICTION]  False
[SAME AUTHOR] False
Current accuracy: 100.00%

Evaluating pair 2/6
Authors: kongbomb, kongbomb
[PREDICTION]  True
[SAME AUTHOR] True
Current accuracy: 100.00%

Evaluating pair 3/6
Authors: kongbomb, kongbomb
[PREDICTION]  True
[SAME AUTHOR] True
Current accuracy: 100.00%

Evaluating pair 4/6
Authors: kongbomb, kongbomb
[PREDICTION]  True
[SAME AUTHOR] True
Current accuracy: 100.00%

Evaluating pair 5/6
Authors: qwerty787788, stolis
[PREDICTION]  False
[SAME AUTHOR] False
Current accuracy: 100.00%

Evaluating pair 6/6
Authors: Lewin, uwi
[PREDICTION]  False
[SAME AUTHOR] False
Current accuracy: 100.00%

Final Accuracy: 100.00%


1.0